In [1]:
import os
import time
import torch
import argparse
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import json
from copy import deepcopy

from models.SASRec import SASRec

from utils.utils import evaluate
from data.MyDataset import MyDataset

def str2bool(s):
    if s not in {'false', 'true'}:
        raise ValueError('Not a valid boolean string')
    return s == 'true'

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='KuaiRand', type=str)
parser.add_argument('--train_dir', default='SASRec', type=str)
parser.add_argument('--model_name', default='SASRec', type=str)
parser.add_argument('--exp_name', default='base', type=str)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--lr', default=0.001, type=float)
parser.add_argument('--maxlen', default=50, type=int)
parser.add_argument('--embed_dim', default=16, type=int)
parser.add_argument('--num_epochs', default=30, type=int)
parser.add_argument('--num_test_neg_item', default=100, type=int)
parser.add_argument('--dropout_rate', default=0.5, type=float)
parser.add_argument('--l2_emb', default=0.0, type=float)
parser.add_argument('--enable_feature_embedding_l2_norm', action='store_true', default=False)
parser.add_argument('--device', default='cuda:0', type=str)
parser.add_argument('--inference_only', default=False, type=str2bool)
parser.add_argument('--state_dict_path', default=None, type=str)
parser.add_argument('--pretrain_model_path', default=None, type=str)
parser.add_argument('--save_freq', default=5, type=int)
parser.add_argument('--val_freq', default=1, type=int)

args = parser.parse_known_args()[0]
save_dir = os.path.join(args.dataset + '_' + args.train_dir, args.exp_name)
if not os.path.isdir(args.dataset + '_' + args.train_dir):
    os.makedirs(args.dataset + '_' + args.train_dir)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'args.txt'), 'a') as f:
    f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + '\n')
    f.write('\n'.join([str(k) + ',' + str(v) for k, v in sorted(vars(args).items(), key=lambda x: x[0])]))
f.close()


In [2]:
# dataset
dataset_train = MyDataset(data_dir='data/' + args.dataset,
                                                max_length=args.maxlen, mode='train', device=args.device)
dataset_valid = MyDataset(data_dir='data/' + args.dataset,
                                                max_length=args.maxlen, mode='val', neg_num=args.num_test_neg_item, device=args.device)
dataset_test = MyDataset(data_dir='data/' + args.dataset,
                                               max_length=args.maxlen, mode='test', neg_num=args.num_test_neg_item, device=args.device)

usernum = dataset_train.user_num
itemnum = dataset_train.item_num
user_features_dim = dataset_train.user_features_dim
item_features_dim = dataset_train.item_features_dim
print('number of users: %d' % usernum, 'number of items: %d' % itemnum)

config = {'embed_dim': args.embed_dim,
          'dim_config': {'item_id': itemnum+1, 'user_id': usernum+1,
                         'item_feature': item_features_dim, 'user_feature': user_features_dim},
          'device': args.device,
          'maxlen': args.maxlen}
dataset_meta_data = json.load(open(os.path.join('data', 'dataset_meta_data.json'), 'r'))
config['item_feature'] = dataset_meta_data[args.dataset]['item_feature']
config['user_feature'] = dataset_meta_data[args.dataset]['user_feature']

if args.model_name == "SASRec":
    model = SASRec(config).to(args.device)
else:
    raise ValueError("model name not supported")
f = open(os.path.join(save_dir, 'log.txt'), 'a')
f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +' model: ' + args.model_name + '\n')

for name, param in model.named_parameters():
    try:
        torch.nn.init.xavier_normal_(param.data)
    except:
        pass  # just ignore those failed init layers

model.train()  # enable model training

epoch_start_idx = 1
if args.state_dict_path is not None:
    try:
        model.load_state_dict(torch.load(args.state_dict_path, map_location=torch.device(args.device)))
        tail = args.state_dict_path[args.state_dict_path.find('epoch=') + 6:]
        epoch_start_idx = int(tail[:tail.find('.')]) + 1
    except:  # in case your pytorch version is not 1.6 etc., pls debug by pdb if load weights failed
        print('failed loading state_dicts, pls check file path: ', end="")
        print(args.state_dict_path)
        print('pdb enabled for your quick check, pls type exit() if you do not need it')
        import pdb

        pdb.set_trace()

if args.inference_only:
    model.eval()
    t_test = evaluate(model, dataset_test, args)
    print('test (NDCG@10: %.4f, HR@10: %.4f)' % (t_test[0], t_test[1]))

bce_criterion = torch.nn.BCEWithLogitsLoss()  # torch.nn.BCELoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.98))

number of users: 27285 number of items: 7583


In [3]:
T = 0.0
t0 = time.time()
best_val_HR = 0.0
best_val_NDCG = 0.0
best_HR = 0.0
best_NDCG = 0.0
best_epoch = -1
best_state_dict = None

for epoch in range(epoch_start_idx, args.num_epochs + 1):
    if args.inference_only: break  # just to decrease identition
    dataloader = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True)
    step = 0
    epoch_loss = 0.0
    train_loop = tqdm(dataloader, desc="Training Progress")
    for data in train_loop:
        step += 1
        user_id, history_items, history_items_len, target_item_id, \
            user_features, item_features, label, cold_item = data

        logits = model(user_id, target_item_id, history_items, history_items_len, user_features, item_features)
        if args.model_name == "CB2CF":
            logits, loss_mse = logits

        adam_optimizer.zero_grad()
        loss = bce_criterion(logits, label)

        if 'item_embedding' in model.state_dict().keys():
            for param in model.item_embedding.parameters():
                loss += args.l2_emb * torch.norm(param)
        if 'user_embedding' in model.state_dict().keys():
            for param in model.user_embedding.parameters():
                loss += args.l2_emb * torch.norm(param)
        if args.enable_feature_embedding_l2_norm:
            for name in model.state_dict().keys():
                if 'item_fm_2nd_order_sparse_emb' in name or 'user_fm_2nd_order_sparse_emb' in name:
                    loss += args.l2_emb * torch.norm(model.state_dict()[name])

        if args.model_name == "CB2CF":
            loss +=  loss_mse * args.CB2CF_alpha

        loss.backward()
        adam_optimizer.step()
        epoch_loss += loss.item()
        train_loop.set_description("Epoch {}/{}".format(epoch, args.num_epochs))
        train_loop.set_postfix(loss=epoch_loss/step)

    print("Epoch: {}, loss: {}".format(epoch, epoch_loss / step))


    if epoch % args.val_freq == 0:
        model.eval()
        t1 = time.time() - t0
        T += t1
        print('Evaluating', end='')
        t_test = evaluate(model, dataset_test, args)
        t_valid = evaluate(model, dataset_valid, args)
        print('epoch:%d, time: %f(s), valid (NDCG@10: %.4f, HR@10: %.4f), test (NDCG@10: %.4f, HR@10: %.4f)'
              % (epoch, T, t_valid[0], t_valid[1], t_test[0], t_test[1]))

        if t_valid[1] > best_val_HR:
            best_val_HR = t_valid[1]
            best_HR = t_test[1]
            best_NDCG = t_test[0]
            best_epoch = epoch
            best_state_dict = deepcopy(model.state_dict())

        f.write(str(t_valid) + ' ' + str(t_test) + '\n')
        f.flush()
        t0 = time.time()
        model.train()

    if epoch % args.save_freq == 0 or epoch == args.num_epochs:
        folder = save_dir
        fname = 'epoch={}.lr={}.embed_dim={}.maxlen={}.l2_emb={}.pth'
        fname = fname.format(epoch, args.lr, args.embed_dim,
                             args.maxlen, args.l2_emb)
        torch.save(model.state_dict(), os.path.join(folder, fname))

f.write("best epoch: {}, best NDCG@10: {}, best HR@10: {}".format(best_epoch, best_NDCG, best_HR) + '\n')
f.close()
print("best epoch: {}, best NDCG@10: {}, best HR@10: {}".format(best_epoch, best_NDCG, best_HR))
torch.save(best_state_dict, os.path.join(save_dir, 'best.pth'))
print("Done")

Epoch 1/30: 100%|██████████| 6720/6720 [06:42<00:00, 16.69it/s, loss=0.594]


Epoch: 1, loss: 0.5943119777810006
Evaluating

Testing Progress: 100%|██████████| 16246/16246 [00:49<00:00, 327.20it/s]


epoch:1, time: 402.757097(s), valid (NDCG@10: 0.2619, HR@10: 0.4493), test (NDCG@10: 0.2475, HR@10: 0.4271)


Epoch 2/30: 100%|██████████| 6720/6720 [06:31<00:00, 17.17it/s, loss=0.475]


...

epoch:29, time: 11564.723861(s), valid (NDCG@10: 0.7236, HR@10: 0.8999), test (NDCG@10: 0.7163, HR@10: 0.8891)


Epoch 30/30: 100%|██████████| 6720/6720 [06:41<00:00, 16.75it/s, loss=0.19] 


Epoch: 30, loss: 0.18965126179003466
Evaluating

Testing Progress: 100%|██████████| 16246/16246 [00:49<00:00, 330.49it/s]

epoch:30, time: 11966.015810(s), valid (NDCG@10: 0.7259, HR@10: 0.8950), test (NDCG@10: 0.7223, HR@10: 0.8923)
best epoch: 24, best NDCG@10: 0.7225888570011661, best HR@10: 0.8928351594238581
Done


In [11]:
from models.DSSM import DSSM, DSSM_SASRec

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='KuaiRand', type=str)
parser.add_argument('--train_dir', default='DSSM_SASRec', type=str)
parser.add_argument('--model_name', default='DSSM_SASRec', type=str)
parser.add_argument('--exp_name', default='base', type=str)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--lr', default=0.001, type=float)
parser.add_argument('--maxlen', default=50, type=int)
parser.add_argument('--embed_dim', default=16, type=int)
parser.add_argument('--num_epochs', default=30, type=int)
parser.add_argument('--num_test_neg_item', default=100, type=int)
parser.add_argument('--dropout_rate', default=0.5, type=float)
parser.add_argument('--l2_emb', default=0.0, type=float)
parser.add_argument('--device', default='cpu', type=str)
parser.add_argument('--inference_only', default=False, type=str2bool)
parser.add_argument('--state_dict_path', default=None, type=str)
parser.add_argument('--pretrain_model_path', default='KuaiRand_SASRec/base/best.pth', type=str)
parser.add_argument('--save_freq', default=5, type=int)
parser.add_argument('--val_freq', default=1, type=int)

args = parser.parse_known_args()[0]
save_dir = os.path.join(args.dataset + '_' + args.train_dir, args.exp_name)
if not os.path.isdir(args.dataset + '_' + args.train_dir):
    os.makedirs(args.dataset + '_' + args.train_dir)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'args.txt'), 'a') as f:
    f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + '\n')
    f.write('\n'.join([str(k) + ',' + str(v) for k, v in sorted(vars(args).items(), key=lambda x: x[0])]))
f.close()

In [12]:
# dataset
dataset_train = MyDataset(data_dir='data/' + args.dataset,
                                                max_length=args.maxlen, mode='train', device=args.device)
dataset_valid = MyDataset(data_dir='data/' + args.dataset,
                                                max_length=args.maxlen, mode='val', neg_num=args.num_test_neg_item, device=args.device)
dataset_test = MyDataset(data_dir='data/' + args.dataset,
                                               max_length=args.maxlen, mode='test', neg_num=args.num_test_neg_item, device=args.device)

usernum = dataset_train.user_num
itemnum = dataset_train.item_num
user_features_dim = dataset_train.user_features_dim
item_features_dim = dataset_train.item_features_dim
print('number of users: %d' % usernum, 'number of items: %d' % itemnum)

config = {'embed_dim': args.embed_dim,
          'dim_config': {'item_id': itemnum+1, 'user_id': usernum+1,
                         'item_feature': item_features_dim, 'user_feature': user_features_dim},
          'device': args.device,
          'maxlen': args.maxlen}
dataset_meta_data = json.load(open(os.path.join('data', 'dataset_meta_data.json'), 'r'))
config['item_feature'] = dataset_meta_data[args.dataset]['item_feature']
config['user_feature'] = dataset_meta_data[args.dataset]['user_feature']

if args.model_name == "DSSM":
    model = DSSM(config).to(args.device)
elif args.model_name == "DSSM_SASRec":
    model = DSSM_SASRec(config).to(args.device)
else:
    raise ValueError("model name not supported")
f = open(os.path.join(save_dir, 'log.txt'), 'a')
f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' model: ' + args.model_name + '\n')

for name, param in model.named_parameters():
    try:
        torch.nn.init.xavier_normal_(param.data)
    except:
        pass  # just ignore those failed init layers

model.train()  # enable model training

epoch_start_idx = 1
if args.state_dict_path is not None:
    try:
        model.load_state_dict(torch.load(args.state_dict_path, map_location=torch.device(args.device)))
        tail = args.state_dict_path[args.state_dict_path.find('epoch=') + 6:]
        epoch_start_idx = int(tail[:tail.find('.')]) + 1
    except:  # in case your pytorch version is not 1.6 etc., pls debug by pdb if load weights failed
        print('failed loading state_dicts, pls check file path: ', end="")
        print(args.state_dict_path)
        print('pdb enabled for your quick check, pls type exit() if you do not need it')
        import pdb

        pdb.set_trace()

if args.model_name == "DSSM_SASRec":
    model.load_and_freeze_backbone(args.pretrain_model_path, True)

if args.inference_only:
    model.eval()
    t_test = evaluate(model, dataset_test, args)
    print('test (NDCG@10: %.4f, HR@10: %.4f)' % (t_test[0], t_test[1]))

bce_criterion = torch.nn.BCEWithLogitsLoss()  # torch.nn.BCELoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.98))

number of users: 27285 number of items: 7583


In [13]:
T = 0.0
t0 = time.time()
best_val_HR = 0.0
best_val_NDCG = 0.0
best_HR = 0.0
best_NDCG = 0.0
best_epoch = -1
best_state_dict = None

for epoch in range(epoch_start_idx, args.num_epochs + 1):
    if args.inference_only: break  # just to decrease identition
    dataloader = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True)
    step = 0
    epoch_loss = 0.0
    train_loop = tqdm(dataloader, desc="Training Progress")
    for data in train_loop:
        step += 1
        user_id, history_items, history_items_len, target_item_id, \
            user_features, item_features, label, cold_item = data

        logits = model(user_id, target_item_id, history_items, history_items_len, user_features, item_features)

        adam_optimizer.zero_grad()

        loss = bce_criterion(logits, label)
        if 'item_embedding' in model.state_dict().keys():
            for param in model.item_embedding.parameters():
                loss += args.l2_emb * torch.norm(param)
        if 'user_embedding' in model.state_dict().keys():
            for param in model.user_embedding.parameters():
                loss += args.l2_emb * torch.norm(param)

        loss.backward()
        adam_optimizer.step()
        epoch_loss += loss.item()
        train_loop.set_description("Epoch {}/{}".format(epoch, args.num_epochs))
        train_loop.set_postfix(loss=epoch_loss/step)
    print("Epoch: {}, loss: {}".format(epoch, epoch_loss / step))

    if epoch % args.val_freq == 0:
        model.eval()
        t1 = time.time() - t0
        T += t1
        print('Evaluating', end='')
        t_test = evaluate(model, dataset_test, args)
        t_valid = evaluate(model, dataset_valid, args)
        print('epoch:%d, time: %f(s), valid (NDCG@10: %.4f, HR@10: %.4f), test (NDCG@10: %.4f, HR@10: %.4f)'
              % (epoch, T, t_valid[0], t_valid[1], t_test[0], t_test[1]))

        if t_valid[1] > best_val_HR:
            best_val_HR = t_valid[1]
            best_HR = t_test[1]
            best_NDCG = t_test[0]
            best_epoch = epoch
            best_state_dict = deepcopy(model.state_dict())

        f.write(str(t_valid) + ' ' + str(t_test) + '\n')
        f.flush()
        t0 = time.time()
        model.train()

    if epoch % args.save_freq == 0 or epoch == args.num_epochs:
        folder = save_dir
        fname = 'epoch={}.lr={}.embed_dim={}.maxlen={}.l2_emb={}.pth'
        fname = fname.format(epoch, args.lr, args.embed_dim,
                             args.maxlen, args.l2_emb)
        torch.save(model.state_dict(), os.path.join(folder, fname))

f.write("best epoch: {}, best NDCG@10: {}, best HR@10: {}".format(best_epoch, best_NDCG, best_HR) + '\n')
f.close()
print("best epoch: {}, best NDCG@10: {}, best HR@10: {}".format(best_epoch, best_NDCG, best_HR))
torch.save(best_state_dict, os.path.join(save_dir, 'best.pth'))
print("Done")

Epoch 1/30: 100%|██████████| 6720/6720 [05:13<00:00, 21.44it/s, loss=2.83]


Epoch: 1, loss: 2.829629444432933
Evaluating

Testing Progress: 100%|██████████| 16246/16246 [01:29<00:00, 180.80it/s]


epoch:1, time: 313.456668(s), valid (NDCG@10: 0.2922, HR@10: 0.4978), test (NDCG@10: 0.2865, HR@10: 0.4890)


Epoch 2/30: 100%|██████████| 6720/6720 [05:04<00:00, 22.08it/s, loss=0.399]


...

Epoch: 28, loss: 0.14797178107934694
Evaluating

Testing Progress: 100%|██████████| 16246/16246 [01:26<00:00, 187.07it/s]


epoch:28, time: 8654.148785(s), valid (NDCG@10: 0.6293, HR@10: 0.8483), test (NDCG@10: 0.6498, HR@10: 0.8619)


Testing Progress: 100%|██████████| 16246/16246 [01:35<00:00, 170.49it/s]

epoch:30, time: 9232.966684(s), valid (NDCG@10: 0.6336, HR@10: 0.8472), test (NDCG@10: 0.6525, HR@10: 0.8617)
best epoch: 7, best NDCG@10: 0.7431923423428468, best HR@10: 0.910931921703804
Done


In [22]:
from models.DSSM import DSSM_PTCR, DSSM_SASRec_PTCR
from utils.utils import evaluate_prompt
from data.MyDataset import PTCRDataset

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='KuaiRand', type=str)
parser.add_argument('--train_dir', default='DSSM_SASRec_PTCR', type=str)
parser.add_argument('--model_name', default='DSSM_SASRec_PTCR', type=str)
parser.add_argument('--exp_name', default='base', type=str)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--lr', default=0.0005, type=float)
parser.add_argument('--maxlen', default=50, type=int)
parser.add_argument('--embed_dim', default=16, type=int)
parser.add_argument('--num_epochs', default=10, type=int)
parser.add_argument('--num_test_neg_item', default=100, type=int)
parser.add_argument('--dropout_rate', default=0.5, type=float)
parser.add_argument('--l2_emb', default=0.0, type=float)
parser.add_argument('--device', default='cpu', type=str)
parser.add_argument('--inference_only', default=False, type=str2bool)
parser.add_argument('--state_dict_path', default=None, type=str)
parser.add_argument('--pretrain_model_path', default='KuaiRand_DSSM_SASRec/base/best.pth', type=str)
parser.add_argument('--alpha', default=0.01, type=float)
parser.add_argument('--beta', default=0.01, type=float)
parser.add_argument('--save_freq', default=5, type=int)
parser.add_argument('--val_freq', default=1, type=int)

args = parser.parse_known_args()[0]
save_dir = os.path.join(args.dataset + '_' + args.train_dir, args.exp_name)
if not os.path.isdir(args.dataset + '_' + args.train_dir):
    os.makedirs(args.dataset + '_' + args.train_dir)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'args.txt'), 'a') as f:
    f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + '\n')
    f.write('\n'.join([str(k) + ',' + str(v) for k, v in sorted(vars(args).items(), key=lambda x: x[0])]))
f.close()

In [23]:
# dataset
dataset_train = PTCRDataset(data_dir='data/' + args.dataset,
                            max_length=args.maxlen, mode='train', device=args.device)
dataset_valid = PTCRDataset(data_dir='data/' + args.dataset,
                            max_length=args.maxlen, mode='val', neg_num=args.num_test_neg_item, device=args.device)
dataset_test = PTCRDataset(data_dir='data/' + args.dataset,
                           max_length=args.maxlen, mode='test', neg_num=args.num_test_neg_item, device=args.device)

usernum = dataset_train.user_num
itemnum = dataset_train.item_num
user_features_dim = dataset_train.user_features_dim
item_features_dim = dataset_train.item_features_dim
print('number of users: %d' % usernum, 'number of items: %d' % itemnum)

config = {'embed_dim': args.embed_dim,
          'dim_config': {'item_id': itemnum + 1, 'user_id': usernum + 1,
                         'item_feature': item_features_dim, 'user_feature': user_features_dim},
          'prompt_embed_dim': args.embed_dim,
          'prompt_net_hidden_size': args.embed_dim,
          'device': args.device,
          'maxlen': args.maxlen
          }
dataset_meta_data = json.load(open(os.path.join('data', 'dataset_meta_data.json'), 'r'))
config['item_feature'] = dataset_meta_data[args.dataset]['item_feature']
config['user_feature'] = dataset_meta_data[args.dataset]['user_feature']

if args.model_name == "DSSM_PTCR":
    model = DSSM_PTCR(config).to(args.device)
elif args.model_name == "DSSM_SASRec_PTCR":
    model = DSSM_SASRec_PTCR(config).to(args.device)
else:
    raise Exception("No such model!")

f = open(os.path.join(save_dir, 'log.txt'), 'a')
f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' model: ' + args.model_name + '\n')

for name, param in model.named_parameters():
    try:
        torch.nn.init.xavier_normal_(param.data)
    except:
        pass  # just ignore those failed init layers

model.train()  # enable model training

epoch_start_idx = 1
if args.state_dict_path is not None:
    try:
        model.load_state_dict(torch.load(args.state_dict_path, map_location=torch.device(args.device)))
        tail = args.state_dict_path[args.state_dict_path.find('epoch=') + 6:]
        epoch_start_idx = int(tail[:tail.find('.')]) + 1
    except:  # in case your pytorch version is not 1.6 etc., pls debug by pdb if load weights failed
        print('failed loading state_dicts, pls check file path: ', end="")
        print(args.state_dict_path)
        print('pdb enabled for your quick check, pls type exit() if you do not need it')
        import pdb

        pdb.set_trace()

# 加载 backbone
model.load_and_freeze_backbone(args.pretrain_model_path)

if args.inference_only:
    model.eval()
    t_test = evaluate_prompt(model, dataset_test, args)
    print('test (NDCG@10: %.4f, HR@10: %.4f)' % (t_test[0], t_test[1]))

bce_criterion = torch.nn.BCEWithLogitsLoss()  # torch.nn.BCELoss()
# backbone冻结参数不参与训练
adam_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.lr,
                                  betas=(0.9, 0.98))

number of users: 27285 number of items: 7583


In [25]:
from torch.nn import functional as F

T = 0.0
t0 = time.time()
best_val_HR = 0.0
best_val_NDCG = 0.0
best_HR = 0.0
best_NDCG = 0.0
best_epoch = -1
best_state_dict = None

for epoch in range(epoch_start_idx, args.num_epochs + 1):
    if args.inference_only: break  # just to decrease identition
    dataloader = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True)
    step = 0
    epoch_loss = 0.0
    train_loop = tqdm(dataloader, desc="Training Progress")

    for data in train_loop:
        step += 1
        user_id, history_items, history_items_len, target_item_id, \
        user_features, item_features, label, cold_item, \
        item_pos_feedback, item_pos_feedback_len, item_neg_feedback, item_neg_feedback_len = data

        logits, loss_pfpe = model(user_id, target_item_id, history_items, history_items_len, user_features,
                                  item_features,
                                  item_pos_feedback, item_pos_feedback_len, item_neg_feedback,
                                  item_neg_feedback_len)
        adam_optimizer.zero_grad()
        loss = bce_criterion(logits, label)
        loss += args.alpha * loss_pfpe.sum(dim=1).mean(dim=0)

        # fape loss
        selected_indices = (label == 1) & (cold_item == 1)
        pos_cold_item_logits = logits[selected_indices]
        selected_indices = (label == 0) & (cold_item == 0)
        neg_hot_item_logits = logits[selected_indices]
        loss_fape = F.softplus(-(pos_cold_item_logits.sum() * len(neg_hot_item_logits) -
                                 neg_hot_item_logits.sum() * len(pos_cold_item_logits)), beta=1, threshold=10)

        loss += args.beta * loss_fape

        loss.backward()
        adam_optimizer.step()
        epoch_loss += loss.item()
        train_loop.set_description("Epoch {}/{}".format(epoch, args.num_epochs))
        train_loop.set_postfix(loss=epoch_loss/step)

    print("Epoch: {}, loss: {}".format(epoch, epoch_loss / step))

    if epoch % args.val_freq == 0:
        model.eval()
        t1 = time.time() - t0
        T += t1
        print('Evaluating', end='')
        t_valid = evaluate_prompt(model, dataset_valid, args, 'val')
        t_test = evaluate_prompt(model, dataset_test, args, 'test')
        print('epoch:%d, time: %f(s), valid (NDCG@10: %.4f, HR@10: %.4f), test (NDCG@10: %.4f, HR@10: %.4f)'
              % (epoch, T, t_valid[0], t_valid[1], t_test[0], t_test[1]))

        if t_valid[1] > best_val_HR:
            best_val_HR = t_valid[1]
            best_HR = t_test[1]
            best_NDCG = t_test[0]
            best_epoch = epoch
            best_state_dict = deepcopy(model.state_dict())

        f.write(str(t_valid) + ' ' + str(t_test) + '\n')
        f.flush()
        t0 = time.time()
        model.train()

    if epoch % args.save_freq == 0 or epoch == args.num_epochs:
        folder = save_dir
        fname = 'epoch={}.lr={}.embed_dim={}.maxlen={}.alpha={}.beta={}.pth'
        fname = fname.format(epoch, args.lr, args.embed_dim,
                             args.maxlen, args.alpha, args.beta)
        torch.save(model.state_dict(), os.path.join(folder, fname))
        torch.save(best_state_dict, os.path.join(save_dir, 'best.pth'))

f.write("best epoch: {}, best NDCG@10: {}, best HR@10: {}".format(best_epoch, best_NDCG, best_HR) + '\n')
f.close()
print("best epoch: {}, best NDCG@10: {}, best HR@10: {}".format(best_epoch, best_NDCG, best_HR))
torch.save(best_state_dict, os.path.join(save_dir, 'best.pth'))
print("Done")


Epoch 1/10: 100%|██████████| 6720/6720 [05:45<00:00, 19.45it/s, loss=0.256]


Epoch: 1, loss: 0.2561366929295695
Evaluating

Testing Progress: 100%|██████████| 16246/16246 [01:44<00:00, 155.65it/s]


epoch:1, time: 345.464198(s), valid (NDCG@10: 0.7352, HR@10: 0.9013), test (NDCG@10: 0.7430, HR@10: 0.9091)


Epoch 2/10: 100%|██████████| 6720/6720 [06:29<00:00, 17.24it/s, loss=0.17] 


...

epoch:9, time: 3142.653582(s), valid (NDCG@10: 0.3895, HR@10: 0.6221), test (NDCG@10: 0.3778, HR@10: 0.6090)


Epoch 10/20: 100%|██████████| 6720/6720 [05:53<00:00, 19.01it/s, loss=0.135]


Epoch: 10, loss: 0.13539918167738332
Evaluating

Testing Progress: 100%|██████████| 16246/16246 [01:43<00:00, 156.32it/s]


epoch:10, time: 3496.210682(s), valid (NDCG@10: 0.2891, HR@10: 0.5161), test (NDCG@10: 0.2741, HR@10: 0.4906)
best epoch: 4, best NDCG@10: 0.813525013478219, best HR@10: 0.9213344823341130
Done
